# Qiskit Runtime Servies
## Executando vários circuitos com um único job

In [1]:
import time
import numpy as np
from qiskit import *
from qiskit.circuit import Parameter
from qiskit.quantum_info import Statevector, Pauli, SparsePauliOp
from qiskit.circuit.library import RealAmplitudes
import matplotlib.pyplot as plt
import matplotlib.ticker as tck
from qiskit.tools.visualization import plot_histogram

In [3]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options, Sampler, Estimator

In [5]:
QiskitRuntimeService.save_account(channel='ibm_quantum', 
                                  token='e1d110334509c3bb9daa4b0be80028bf82dddd70b17b73e7af3e8c4540bb1dd84e268d68a80efc238061afa9a5f516714392133c7ae8a1d2f148b7bdf72637e6',
                                  overwrite=True)

In [6]:
service = QiskitRuntimeService(channel='ibm_quantum')

In [7]:
service.backends()

[<IBMBackend('ibmq_qasm_simulator')>,
 <IBMBackend('ibmq_armonk')>,
 <IBMBackend('ibmq_santiago')>,
 <IBMBackend('ibmq_bogota')>,
 <IBMBackend('ibmq_lima')>,
 <IBMBackend('ibmq_belem')>,
 <IBMBackend('ibmq_quito')>,
 <IBMBackend('simulator_statevector')>,
 <IBMBackend('simulator_mps')>,
 <IBMBackend('simulator_extended_stabilizer')>,
 <IBMBackend('simulator_stabilizer')>,
 <IBMBackend('ibmq_jakarta')>,
 <IBMBackend('ibmq_manila')>,
 <IBMBackend('ibm_lagos')>,
 <IBMBackend('ibm_nairobi')>,
 <IBMBackend('ibm_perth')>,
 <IBMBackend('ibm_oslo')>]

In [33]:
backend = service.backends(simulator=True)[0]
print(backend)

<IBMBackend('ibmq_qasm_simulator')>


In [10]:
def bernstein_vazirani(string):
    string_length = len(string)
    qc = QuantumCircuit(string_length+1, string_length)
    qc.h(range(string_length))
    qc.x(string_length)
    qc.h(string_length)
    for ii, yesno in enumerate(reversed(string)): # the black box
        if yesno == '1':
            qc.cx(ii, string_length)
    qc.h(range(string_length))
    qc.measure(range(string_length), range(string_length))
    return qc
qc1 = bernstein_vazirani('111')
qc2 = bernstein_vazirani('000')

In [12]:
options = Options(simulator={"seed_simulator": 42}, resilience_level=0) # Do not change values in simulator
with Session(service=service, backend=backend):
    sampler = Sampler(options=options)
    job = sampler.run(circuits=[qc1,qc2])

In [27]:
job.result().quasi_dists

[{7: 1.0}, {0: 1.0}]

In [40]:
backend = service.backends(simulator=False)[2]
print(backend)

<IBMBackend('ibmq_bogota')>


In [41]:
with Session(service=service, backend=backend):
    sampler = Sampler(options=options)
    job = sampler.run(circuits=[qc1,qc2])

IBMRuntimeError: 'Failed to run program: "HTTPSConnectionPool(host=\'runtime-us-east.quantum-computing.ibm.com\', port=443): Max retries exceeded with url: /jobs (Caused by ResponseError(\'too many 503 error responses\'))"'